# Trabajo Final: Deep Learning haciendo uso de CNN

**Docente**: 
- Richard Fernández

**Integrantes**:
- Luis Parodi
- Eduardo Contreras
- Dante Moreno

## 1. Introducción

## 2. Marco Teórico

### 2.1. Redes Neuronales Convolucionales

### 2.2. Aprendizaje Residual

El uso de redes neuronales convolucionales ha generado un gran avance en el estudio de visión computacional. Aunque crear redes más profundas, es decir, con más capas, podría parecer una forma de mejorar los resultados, en realidad vuelve más díficil el entrenamiento. Esto es llamado un problema de degradación: conforme la profundidad de la red aumenta, su resultado en accuracy disminuye rápidamente$^{[1]}$.

### 2.3. Redes Residuales

## 3. Caso de Estudio

In [ ]:
import torch
import torch.nn as nn

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000)

### Resultados del modelo

## 4. Conclusiones

## 5. Recomendaciones

## Referencias bibliográficas:

* [1] He, K., Zhang, X., Ren, S., & Sun, J. (2015). *Deep Residual Learning for Image Recognition* Recuperado de: https://arxiv.org/abs/1512.03385
* [2] Feng, V. (15 de julio de 2020). An Overview of ResNet and its Variants. *Towards Data Science*. Recuperado de: https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035
* [3] Mutjaba, H. (28 de septiembre de 2020). Introduction to Resnet or Residual Network. *Great Learning*. Recuperado de: https://www.mygreatlearning.com/blog/resnet/
* [4] Mandal, M. (1 de mayo de 2021). Introduction to Convolutional Neural Networks (CNN). *Analytics Vidha*. Recuperado de: https://www.analyticsvidhya.com/blog/2021/05/convolutional-neural-networks-cnn/ 